In [25]:
import pinocchio as pin
from pinocchio.utils import *

import plotly.express as px

In [5]:
model = pin.buildModelFromUrdf("robot_utils/panda-model/panda_arm.urdf")
data = model.createData()



In [57]:
q0 = np.array([1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0])
v0 = np.zeros(7)
a0 = np.ones(7)*10

model.gravity = pinocchio.Motion.Zero()

lr = 1e-3
loss = []
for i in range(10000):

    _, _, _ = pin.computeRNEADerivatives(model,data,q0,v0,a0)

    loss.append(np.linalg.norm(data.tau))

    dtau_dq = data.dtau_dq # Derivatives of the ID w.r.t. the joint config vector
    dtau_dv = data.dtau_dv # Derivatives of the ID w.r.t. the joint velocity vector
    dtau_da = data.M # Derivatives of the ID w.r.t. the joint acceleration vector

    a0 -= 210*lr*dtau_da.T@data.tau;

    
print(a0)
px.line(loss)

[ 0.093  0.211  0.47   0.218  3.767  3.677 10.246]


In [55]:
_, _, _ = pin.computeRNEADerivatives(model,data,np.array([1.0, -1.0, 0.5, -2.0, 1.0, 2.0, -0.5]),np.zeros(7),np.array([10, 10, 10, 10, 10, 10, 10]))
np.set_printoptions(precision=3, suppress=True)
print(dtau_da)


[[ 0.639 -0.573  0.144  0.365  0.014 -0.014 -0.006]
 [-0.573  1.738 -0.338 -0.779  0.01  -0.036  0.008]
 [ 0.144 -0.338  1.182 -0.005 -0.039 -0.097 -0.006]
 [ 0.365 -0.779 -0.005  0.819  0.013  0.056 -0.007]
 [ 0.014  0.01  -0.039  0.013  0.024 -0.001  0.001]
 [-0.014 -0.036 -0.097  0.056 -0.001  0.033  0.   ]
 [-0.006  0.008 -0.006 -0.007  0.001  0.     0.006]]
